## SpiceyPy

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [3]:
import spiceypy as spice

In [4]:
# Print out the toolkit version
spice.tkvrsn("TOOLKIT")

'CSPICE_N0067'

## Kernels

[SPICE Kernel Required Reading](https://naif.jpl.nasa.gov/pub/naif/toolkit_docs/C/req/kernel.html)

SPICE text kernels are:

- text form of planetary constants (text PCK)
- leapseconds (LSK)
- spacecraft clock coefficients (SCLK)
- instrument geometry (IK)
- reference frame specifications  (FK)
- meta-kernels (MK)

SPICE binary kernels:
- ephemeris for vehicles, planents, satellites, comets, asteriods (SPK)
- orientation (attitude) of a spacecraft or other structure (CK)
- special binary form of planetary constants containing only orientation (binary PCK)
- shape models or topographic data for extended objects (DSK)
- mission events (EK)

NAIF kernel naming conventions
- .bc    binary CK
- .bds   binary DSK
- .bes   binary Sequence Component EK
- .bpc   binary PCK
- .bsp   binary SPK
- .tf    text FK
- .ti    text IK
- .tls   text LSK
- .tm    text meta-kernel (FURNSH kernel)
- .tpc   text PCK
- .tsc   text SCLK

In [10]:
from bs4 import BeautifulSoup
from urllib import request, error
import random

In [74]:
def return_table_of_contents(root_url):
    user_agents = [
		'Mozilla/6.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
		'Mozilla/6.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
		'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
		'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
		'Mozilla/6.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
		'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
		'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15'
    ]

    random_agent = random.choice(user_agents)
    naif_kernel_url = root_url
    req_with_headers = request.Request(url=naif_kernel_url, headers={'User-Agent': random_agent})
    naif_html = request.urlopen(req_with_headers).read()
    soup = BeautifulSoup(naif_html, 'html.parser')
    table_of_contents = soup.find('pre')
    return table_of_contents

In [93]:
def returnSubDirs(table_of_contents):
    sub_dirs = []
    for row in table_of_contents.text.split("\n"):
        if len(row) > 0:
            row = row.split("  ")
            row = list(filter(None, row)) # remove empty strings ""
            if "/" in row[0]:
                sub_dirs.append(row[0])
    return sub_dirs

In [105]:
root_url = "https://naif.jpl.nasa.gov/pub/naif/GLL/kernels/"
table_of_contents = return_table_of_contents(root_url)
print(table_of_contents.text)

 Name                    Last modified      Size   Parent Directory                             -   
 aareadme.txt            2004-03-11 14:39  456   
 ck/                     2013-07-12 16:25    -   
 ek/                     2000-07-12 08:40    -   
 ik/                     2012-01-16 06:44    -   
 lsk/                    2005-08-09 17:22    -   
 pck/                    2002-03-01 06:46    -   
 sclk/                   2000-07-20 03:14    -   
 spk/                    2022-02-10 16:07    -   



In [106]:
kernel_types = returnSubDirs(table_of_contents)
print(kernel_types)

[' ck/', ' ek/', ' ik/', ' lsk/', ' pck/', ' sclk/', ' spk/']


In [150]:
def return_kernel_df(kernel_type):
    if kernel_type not in kernel_types:
        print(f"not a valid kernel type, valid kernel options: {kernel_types}")
        return None
    root_url = f"https://naif.jpl.nasa.gov/pub/naif/GLL/kernels/{kernel_type.strip()}"
    table_of_contents = return_table_of_contents(root_url)
    kernel_name = []
    last_modified = []
    for row in table_of_contents.text.split("\n")[1:]:
        row = row.split(" ")
        row = list(filter(None, row)) # remove empty strings ""
        if len(row) > 0:
            #print(row)
            row = [x.strip() for x in row]
            if len(row[0].split(".")) > 1:
                kernel_name.append(row[0])
                last_modified.append(row[1] + " " + row[2])
    import pandas as pd
    kernel_df = pd.DataFrame({"kernel_name": kernel_name, "last_modified": last_modified})
    return kernel_df

In [185]:
kernel_df = return_kernel_df(" ik/")
kernel_df["last_modified"] = pd.to_datetime(kernel_df["last_modified"])
kernel_df = kernel_df.sort_values(by="last_modified")
kernel_df

,kernel_name,last_modified
0,gll24001.ti,2012-01-16 06:44:00
1,gll25001.ti,2012-01-16 06:44:00
2,gll28001.ti,2012-01-16 06:44:00
3,gll29001.ti,2012-01-16 06:44:00
4,gll34001.ti,2012-01-16 06:44:00
5,gll36001.ti,2012-01-16 06:44:00
6,gll37001.ti,2012-01-16 06:44:00


In [186]:
most_recent_date = kernel_df["last_modified"].max()
print(type(most_recent_date))
print(most_recent_date)

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2012-01-16 06:44:00


In [187]:
# return all the kernels that match most recent timestamp
kernel_df[(kernel_df['last_modified'] == most_recent_date)]

,kernel_name,last_modified
0,gll24001.ti,2012-01-16 06:44:00
1,gll25001.ti,2012-01-16 06:44:00
2,gll28001.ti,2012-01-16 06:44:00
3,gll29001.ti,2012-01-16 06:44:00
4,gll34001.ti,2012-01-16 06:44:00
5,gll36001.ti,2012-01-16 06:44:00
6,gll37001.ti,2012-01-16 06:44:00
